In [1]:
import scipy.ndimage.interpolation as interp
from astropy.stats import mad_std
from astropy.stats import sigma_clip
from photutils.utils import calc_total_error
import astropy.stats as stat
from photutils import aperture_photometry, CircularAperture, CircularAnnulus, DAOStarFinder
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.visualization import ZScaleInterval
from astropy.io import fits 
import os 
import glob 
import scipy.ndimage.interpolation as interp
from tqdm import tqdm
from tqdm import tqdm
from importlib import reload
from dred import Calibration
import dred

### Grouping all the Data into their Respective Folders:

In [2]:
#Getting all the files
all_filenames = glob.glob("Checkin1_BiasFlats/*.fits")
all_files = [f.split('/')[-1] for f in all_filenames]

#Sorting the files by type
for f in all_files:
    dred.filesorter(f, 'Flats', 'Twilight Flat V', 'OBJECT', current_path = "Checkin1_BiasFlats")
    dred.filesorter(f, 'Orion_V', '+05', 'RASTRNG', current_path = "Checkin1_BiasFlats")
    dred.filesorter(f, 'Bias', 'Bias Frame', 'OBJECT', current_path = "Checkin1_BiasFlats")
    
#Set your own file paths
path_to_flats = glob.glob('Checkin1_BiasFlats/Flats/*.fits')

path_to_bias = glob.glob('Checkin1_BiasFlats/Bias/*.fits')

path_to_orion = glob.glob('Checkin1_BiasFlats/Orion_V/*.fits')

In [3]:
bias_calibration = Calibration('bias')
bias_calibration.generate_master_bias(path_to_bias)
bias_calibration.apply(path_to_orion, save = True, output_dir = 'Checkin1_BiasFlats/Orion_V/')
bias_calibration.apply(path_to_flats, save = True, output_dir = 'Checkin1_BiasFlats/Flats/')

In [5]:
# Grabbing all the bias subtracted science frames
b_sub_flats = glob.glob('Checkin1_BiasFlats/Flats/b_*.fits')

In [7]:
flat_calibration = Calibration('flats')

In [8]:
flat_calibration.generate_master_flat(b_sub_flats)

TypeError: unsupported operand type(s) for /: 'MasterImage' and 'int'